## このカーネルについて

2019/03開催[Kaggleタイタニックハンズオン（サポーターズ勉強会 オンライン）](https://talent.supporterz.jp/events/2992699f-53ed-417b-abf7-7d7bba931037/)用のkernelです。  
ハンズオンパートで使います

1. 単純なモデルを作成する
2. モデル作成を一緒に一回やって流れをつかむ
3. 性能が上がると考える選択肢を試す（サンプルコードのブロックを実行）

## 基本操作

コードが書かれているブロックを **セル** と呼びます。  

1. セルをクリックして選択する（入力できる状態になる）
2. 選択したセルを実行する
    - 入力できる状態でShift+Enterキーを押す（こちらに慣れると簡単です）
    - 左側に表示される再生ボタン▶をクリック

In [ ]:
# 前処理に必要なモジュールの読み込み
import numpy as np
import pandas as pd

タイタニックコンペでは以下のフォルダ配置となる。これを踏まえて、相対パスでデータを指定する

- ┣ ディレクトリ
    - ┗ 現在のkernel
- ┗ input/
    - ┣ train.csv（モデル作成用データ）
    - ┣ test.csv（予測対象データ）
    - ┗ gender_submission.csv（提出練習用データ）

In [ ]:
# 読み込んだデータはExcelの表のような形式で扱う（行と列がある）
# モデル作成用データの読み込み（生存か死亡か知っているデータ）
train_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/train.csv')
# 予測対象データの読み込み（生存か死亡か知らないデータ）
test_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/test.csv')

`read_csv`  
解説: https://note.nkmk.me/python-pandas-read-csv-tsv/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
# モデル作成用データのサイズを確認
# (行数, 列数) で表示される
train_df.shape

In [ ]:
# 予測対象データのサイズを確認
# モデル作成用データに対して1列少ない
test_df.shape

`shape`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html

In [ ]:
# モデル作成用データの上から5行を表示
# 参考: train_df.head(7) # 上から7行表示
train_df.head()

In [ ]:
# 予測対象データの上から5行を表示
# Survivedの列（生存か死亡かを表す）がないことが確認できる
test_df.head()

`head`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [ ]:
# モデル作成用データの情報を確認
train_df.info()

In [ ]:
# 予測対象データの情報を確認
test_df.info()

`info`  
解説: https://note.nkmk.me/python-pandas-len-shape-size/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html

| infoによる情報 | 意味 |
| ----|---- | 
| int | 整数 |
| float | 浮動小数点数 |
| object | 文字列 |

| データの情報 | 列名 |　意味 |
| ----- | ----- | ----- |
| int | PassengerId | 乗客ID |
| int | **Pclass** | チケットの等級 (1, 2, 3) |
| int | SibSp | 同乗した兄弟姉妹/配偶者の人数 |
| int | Parch | 同乗した両親/子供の人数 |
| int | *Survived* | 0：死亡、1：生存 |
| float | **Age** | 年齢（推測があるため、浮動小数点数） |
| float | Fare | 運賃 |
| object | Name | 氏名 |
| object | **Sex** | 性別 |
| object | Ticket | チケット番号 |
| object | Cabin | 船室番号　|
| object | **Embarked** | 乗船した港の頭文字(S, Q, C) |

ref: https://www.kaggle.com/c/titanic/data

## 1. 単純なモデル

性別を元に生死を予測するモデルを作成する

- 男性：死亡
- 女性：生存

注：この単純なモデルは機械学習で作るのではなく、人が決めたルールを実装して実現

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_sex.png)

性別ごとの生死のヒストグラムから

- 男性の生存率は低い（左側より右側のほうが低い）
- 女性の生存率は高い（左側よりも右側のほうが高い）
- → 性別は生死を予測する際に有力な情報と考えられる

※生死が取りうる値は0か1。区間は2つなので、0の数と1の数に分かれる

In [ ]:
# モデル作成・性能評価に使うモジュールの読み込み
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# モデルが予測に使う情報（ここでは性別）をx, モデルが予測する情報（ここでは生死）をyとする
x = train_df['Sex']
y = train_df['Survived']

参考: 列の取り出し https://note.nkmk.me/python-pandas-index-row-column/  
ドキュメント: 10 minutes pandasの中のSelection https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#selection

In [ ]:
# 引数の辞書のキー（コロンの左側）に一致する要素が、辞書の値（コロンの右側）に置き換わる（femaleが1に置き換わり、maleが0に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
# astype(int)でデータの型が文字列から整数へ変わることに対応
y_pred = x.map({'female': 1, 'male': 0}).astype(int)

`map`  
参考: https://note.nkmk.me/python-pandas-map-replace/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html

タイタニックの場合、モデルの性能はaccuracyというスコアで評価される  
（**注**：性能を表すスコアは他にもある）

- 418名のうち生死を正しく予想できたものの割合
- 1に近いほど性能がよい

| ケース | 正解／誤り |
| ----- | ----- |
| 生存した乗客を生存と予測 | 正解 |
| 生存した乗客を死亡と予測 | 誤り |
| 死亡した乗客を生存と予測 | 誤り |
| 死亡した乗客を死亡と予測 | 正解 |

accuracy = (正解の総数) / (正解の総数 + 誤りの総数)

>Your score is the percentage of passengers you correctly predict. This is known simply as "accuracy”.

ref: https://www.kaggle.com/c/titanic#evaluation

In [ ]:
# accuracyを算出して表示
accuracy_score(y, y_pred)

`accuracy_score`  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [ ]:
x_test = test_df['Sex']
# 予測対象データについて生死を予測（Sexの値を元に、生死を予測）
y_test_pred = x_test.map({'female': 1, 'male': 0}).astype(int)

- 予測した結果から提出用データを作る
- 提出用データは以下の形式のCSVとするように指定されている（1が生存、0が死亡）

PassengerId | Survived
----- | -----
892 | 0
: | :
1309|1

- ref: https://www.kaggle.com/c/titanic#evaluation

In [ ]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': y_test_pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

## 2. モデル作成を一緒に体験

単純なモデルよりも性能のいいモデル作りに挑戦

### SexとPclassからモデル作成

In [ ]:
# SexとPclassから生死を予測するモデルを作ることにする
columns = ['Pclass', 'Sex']

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_pclass.png)

- Pclass=1は生存者が過半数を超えている
- Pclass=2は生存者が半分程度
- Pclass=3は生存者が少ない（4分の1程度）
- →Pclassも生死の予測に使える

In [ ]:
# モデルが予測に使う情報をX, モデルが予測する情報（ここでは生死）をyとする（Xとyという変数名が多い）
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [ ]:
X.head()

- いくつかの文字列の値を取る変数：カテゴリ変数
    - Sex: male, female
- 文字列を整数に変換する
    - Sex: male=1, female=0として置き換え

In [ ]:
# 性別（female/male）を0/1に変換する（maleとfemaleのままではsklearnが扱えない）
# カテゴリを整数に置き換えるための辞書を用意
gender_map = {'female': 0, 'male': 1}
# 引数の辞書のキー（コロンの左側）に一致する要素が、辞書の値（コロンの右側）に置き換わる（femaleが0に置き換わり、maleが1に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [ ]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

`drop`  
参考: https://note.nkmk.me/python-pandas-drop/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
# 前処理したモデル作成用データの確認
X.head()

このあとモデルを作成するが、予測対象データについて生死を予測する前に、  
どの程度の性能のモデルなのか確認したい。  
→モデル作成用データをランダムに2つに分ける（`train_test_split`）

- モデル作成用データのうち、例えば7割でモデルを作る
- 残りの3割でモデルの性能を評価する

参考情報

- ランダムに分けているが、`random_state`引数の値が同じなら、**何回実行しても同じ** ようにランダムに分かれる

In [ ]:
# 今回のハンズオンは7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

`train_test_split`  
参考: https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

| 変数名 | 用途 |
| ----- | ----- |
| X_train | モデル作成用のデータ（乗客の属性） |
| y_train | モデル作成用のデータ（生死） |
| X_val | モデルの性能確認用のデータ（乗客の属性） |
| y_val | モデルの性能確認用のデータ（生死） |

In [ ]:
# モデル作成用のデータの数の確認
len(y_train)

In [ ]:
# モデル性能確認用のデータの数の確認
len(y_val)

In [ ]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

参考: https://scikit-learn.org/stable/modules/tree.html#classification  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
# モデル性能確認用データについて生死を予測
pred = model.predict(X_val)
# accuracyを算出して表示
accuracy_score(y_val, pred)

別のモデルを作ったときに、以前に作ったモデルよりスコアが低ければ、提出しないという判断ができる  
（Kaggleに提出しなくても試行錯誤が進められる。提出回数には日次の上限がある）

In [ ]:
# 提出する場合は以下の2つのセルを実行
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [ ]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

## 3. 用意した選択肢を試す

欠損値の対応を新しく扱う

### （選択肢1）Ageを追加

In [ ]:
# モデルの予測に使う情報にAgeを追加
columns = ['Pclass', 'Sex', 'Age']

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/age_hist_stacked_survived.png)

0〜80歳を10分割なので、約8歳ずつ分割している  
助かっている割合が高い年齢層は0歳〜8歳（半数以上）、10代前半、30代後半、50〜60歳（半数程度）

In [ ]:
# モデルが予測に使う情報をX, モデルが予測する情報（ここでは生死）をyとする（Xとyという変数名が多い）
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [ ]:
X.head()

In [ ]:
# モデル作成用データの欠損値の確認
X.isnull().sum()

In [ ]:
# 予測対象データの欠損値の確認
X_test.isnull().sum()

`isnull().sum()`  
参考: https://note.nkmk.me/python-pandas-nan-judge-count/

Ageがモデル作成用・性能評価用とで20%程度欠けている

- 欠けたデータ＝**欠損値**（missing value、欠測値とも呼ばれる）  
- 欠損の要因：データの収集過程で抜けてしまったなど
- 機械学習のツールは **一般に欠損値に対処できない**（そのためデータを前処理する）
- 削除すると貴重なデータが減るので、**埋める**

モデル作成用データの平均値（=年齢の総和/個数）で埋める

In [ ]:
# Ageの欠損を平均値で埋めた列AgeFillを追加
age_mean = X['Age'].mean()
print(f'Age mean: {age_mean}')
X['AgeFill'] = X['Age'].fillna(age_mean)
X_test['AgeFill'] = X_test['Age'].fillna(age_mean)

`mean`  
参考: https://deepage.net/features/pandas-mean.html  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.mean.html

`fillna`  
参考: https://note.nkmk.me/python-pandas-nan-dropna-fillna/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [ ]:
# 欠損を含むAge列を削除（年齢の情報はAgeFill列を参照する）
X = X.drop(['Age'], axis=1)
X_test = X_test.drop(['Age'], axis=1)

`drop`  
参考: https://note.nkmk.me/python-pandas-drop/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
# モデル作成用データの欠損値の確認
X.isnull().sum()

In [ ]:
# 予測対象データの欠損値の確認
X_test.isnull().sum()

In [ ]:
# 性別（female/male）を0/1に変換する（maleとfemaleのままではsklearnが扱えない）
# カテゴリを整数に置き換えるための辞書を用意
gender_map = {'female': 0, 'male': 1}
# 引数の辞書のキー（コロンの左側）に一致する要素が、辞書の値（コロンの右側）に置き換わる（femaleが1に置き換わり、maleが0に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [ ]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

In [ ]:
# 前処理したモデル作成用データの確認
X.head()

In [ ]:
# 今回のハンズオンではモデル作成用データを7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# モデル作成用のデータの数の確認
len(y_train)

In [ ]:
# モデル性能確認用のデータの数の確認
len(y_val)

In [ ]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

In [ ]:
# モデル性能確認用データについて生死を予測
pred = model.predict(X_val)
# 性能確認：accuracyを算出して表示
accuracy_score(y_val, pred)

In [ ]:
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [ ]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

### （選択肢2）Embarkedを追加

In [ ]:
# モデルの予測に使う情報にEmbarkedを追加
columns = ['Pclass', 'Sex', 'Embarked']

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_embarked.png)

- 港Cは生存者が半分程度
- 港S, Qは生存者が3分の1程度
- →Embarkedを生死の予測に加える

In [ ]:
# モデルが予測に使う情報をX, モデルが予測する情報（ここでは生死）をyとする（Xとyという変数名が多い）
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [ ]:
X.head()

In [ ]:
# モデル作成用データの欠損値の確認
X.isnull().sum()

In [ ]:
# 予測対象データの欠損値の確認
X_test.isnull().sum()

Embarkedがモデル作成用で数件欠けている

*※欠損値の対応の詳細は選択肢1を確認してください*

モデル作成用データで一番多く登場する値（一番多くの人が乗っている港）で埋める

In [ ]:
# Embarkedの欠損値を埋める（欠損値を埋めた後の値で置き換える）
embarked_freq = X['Embarked'].mode()[0]
print(f'Embarked freq: {embarked_freq}')
X['Embarked'] = X['Embarked'].fillna(embarked_freq)
# X_testにEmbarkedの欠損値がないため、実施しない

`mode`  
参考: https://note.nkmk.me/python-pandas-mode/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.mode.html

In [ ]:
# モデル作成用データの欠損値の確認
X.isnull().sum()

In [ ]:
# 性別（female/male）を0/1に変換する（maleとfemaleのままではsklearnが扱えない）
# カテゴリを整数に置き換えるための辞書を用意
gender_map = {'female': 0, 'male': 1}
# 引数の辞書のキー（コロンの左側）に一致する要素が、辞書の値（コロンの右側）に置き換わる（femaleが1に置き換わり、maleが0に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [ ]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

カテゴリ変数Embarkedはダミー変数化する

- S=1, Q=2, C=3と整数に置き換える
    - **本来なかった大小関係が想定されてしまう**
- (S, Q, C)という形式で整数に置き換える

| Embarkedの値 | 置き換えたあと |
| ----- | ----- |
| S | (1, 0, 0) |
| Q | (0, 1, 0) |
| C | (0, 0, 1) |

In [ ]:
# Embarked（S, Q, Cという3カテゴリ）をダミー変数にする
# （Embarked列が消え、Embarked_S, Embarked_Q, Embarked_C列が追加される）
X = pd.get_dummies(X, columns=['Embarked'])
X_test = pd.get_dummies(X_test, columns=['Embarked'])

`get_dummies`  
参考: https://note.nkmk.me/python-pandas-get-dummies/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [ ]:
# 前処理したモデル作成用データの確認
X.head()

In [ ]:
# 今回のハンズオンではモデル作成用データを7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# モデル作成用のデータの数の確認
len(y_train)

In [ ]:
# モデル性能確認用のデータの数の確認
len(y_val)

In [ ]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

In [ ]:
# モデル性能確認用データについて生死を予測
pred = model.predict(X_val)
# 性能確認：accuracyを算出して表示
accuracy_score(y_val, pred)

In [ ]:
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [ ]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

### 練習問題: AgeとEmbarkedの両方を追加

In [ ]:
# train_df, test_dfからSex, Pclass, Age, Embarkedを取り出す

In [ ]:
# Ageの欠損への対応

In [ ]:
# Embarkedの欠損への対応

In [ ]:
# がテゴリ変数Sexへの対応

In [ ]:
# カテゴリ変数Embarkedへの対応

In [ ]:
# モデル作成

In [ ]:
# 性能確認

In [ ]:
# 提出用データ作成

### 変更履歴
https://www.kaggle.com/ftnext/kaggle-spzcolab-201901 をベースにやることを絞ってアップデートした  
ref: https://github.com/PyDataTokyo/pydata-tokyo-tutorial-1